## Day 2: Rock Paper Scissors

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day02/puzzle.ipynb)

Both parts share the same rules of how the score should be calculated. Thus, both parts can share the same domain model and some functions

In [41]:
type Shape = | Rock | Paper | Scissors
type Outcome = | Won | Lost | Draw
type RoundResult = { Opponent: Shape; Me: Shape; Outcome: Outcome}

let shapeScore shape = 
    match shape with
    | Rock -> 1
    | Paper -> 2
    | Scissors -> 3
let outcomeScore outcome = 
    match outcome with
    | Lost -> 0
    | Draw -> 3
    | Won -> 6

let calculateScore (results : RoundResult list) = 
    results
    |> Seq.map (fun {Me = meShape; Outcome = outcome} -> shapeScore meShape + outcomeScore outcome)
    |> Seq.sum


The parts will differ in how `RoundResults` are generated and what the game algorithm input is.

In [42]:
#!value --name sampleRaw

A Y
B X
C Z

In [43]:
#!value --name actualRaw --from-file ./data_actual.txt

### Part 1

In [44]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder
open Farkle.Builder.Regex

#load "../common/common.fsx"
#!share sampleRaw --from value
#!share actualRaw --from value

module Part1 = 
    type RoundStrategy = { Opponent: Shape; Me: Shape }
    type Strategy = RoundStrategy array

    let private opponent = "Opponent" ||= [
        !& "A" => fun () -> Rock
        !& "B" => fun () -> Paper
        !& "C" => fun () -> Scissors
    ]
    let private me = "Me" ||= [
        !& "X" => fun () -> Rock
        !& "Y" => fun () -> Paper
        !& "Z" => fun () -> Scissors
    ]
    let private roundStrategy = "RoundStrategy" ||= [
        !@ opponent .>>. me => fun opponent' me' -> {Opponent = opponent'; Me = me'}
    ]
    let private parser = roundStrategy |> RuntimeFarkle.build |> RuntimeFarkle.parseString

    let private parseRound s = 
        parser s |> Result.get
    let sampleStrategy = Pattern1.read parseRound sampleRaw |> List.ofArray
    let actualStrategy = Pattern1.read parseRound actualRaw |> List.ofArray

Part1.sampleStrategy

Installed Packages Farkle, 6.3.2

index,Opponent,Me
0,Rock,Paper
1,Paper,Rock
2,Scissors,Scissors


In [45]:
module Part1 = 

    let private buildResult ({Opponent = opponentShape; Me = meShape} : Part1.RoundStrategy) outcome =
        { Opponent = opponentShape; Me = meShape; Outcome = outcome }

    let playRound (roundStrategy : Part1.RoundStrategy) : RoundResult = 
        let outcome =
            match roundStrategy with
            | { Opponent = Rock; Me = Rock } -> Draw
            | { Opponent = Rock; Me = Paper } -> Won
            | { Opponent = Rock; Me = Scissors } -> Lost
            | { Opponent = Paper; Me = Rock } -> Lost
            | { Opponent = Paper; Me = Paper } -> Draw
            | { Opponent = Paper; Me = Scissors } -> Won
            | { Opponent = Scissors; Me = Rock } -> Won
            | { Opponent = Scissors; Me = Paper } -> Lost
            | { Opponent = Scissors; Me = Scissors } -> Draw
        buildResult roundStrategy outcome
        

    let rec game (strategy : Part1.RoundStrategy list) : RoundResult list = 
        match strategy with 
        | [] -> []
        | roundStrategy::rest -> 
            (playRound roundStrategy)::(game rest)

In [46]:
let part1SampleResults = Part1.game Part1.sampleStrategy
part1SampleResults

index,Opponent,Me,Outcome
0,Rock,Paper,Won
1,Paper,Rock,Lost
2,Scissors,Scissors,Draw


In [47]:
calculateScore part1SampleResults

15

For the actual input:

In [48]:
Part1.actualStrategy
|> Part1.game 
|> calculateScore

12535

### Part 2

In [49]:
module Part2 = 
    type RoundStrategy = { Opponent: Shape; TargetOutcome: Outcome }
    type Strategy = RoundStrategy array
    let private opponent = "Opponent" ||= [
        !& "A" => fun () -> Rock
        !& "B" => fun () -> Paper
        !& "C" => fun () -> Scissors
    ]
    let private outcome = "Outcome" ||= [
        !& "X" => fun () -> Lost
        !& "Y" => fun () -> Draw
        !& "Z" => fun () -> Won
    ]
    let private roundStrategy = "RoundStrategy" ||= [
        !@ opponent .>>. outcome => fun opponent' outcome' -> {Opponent = opponent'; TargetOutcome = outcome'}
    ]
    let private parser = roundStrategy |> RuntimeFarkle.build |> RuntimeFarkle.parseString

    let private parseRound s = 
        parser s |> Result.get
    let sampleStrategy = Pattern1.read parseRound sampleRaw |> List.ofArray
    let actualStrategy = Pattern1.read parseRound actualRaw |> List.ofArray

Part2.sampleStrategy

index,Opponent,TargetOutcome
0,Rock,Draw
1,Paper,Lost
2,Scissors,Won


In [50]:
module Part2 = 

    let private buildResult ({Opponent = opponentShape; TargetOutcome = outcome} : Part2.RoundStrategy) meShape =
        { Opponent = opponentShape; Me = meShape; Outcome = outcome }

    let playRound (roundStrategy : Part2.RoundStrategy) : RoundResult = 
        let meShape =
            match roundStrategy with
            | { Opponent = Rock; TargetOutcome = Lost } -> Scissors
            | { Opponent = Rock; TargetOutcome = Draw } -> Rock
            | { Opponent = Rock; TargetOutcome = Won } -> Paper
            | { Opponent = Paper; TargetOutcome = Lost } -> Rock
            | { Opponent = Paper; TargetOutcome = Draw } -> Paper
            | { Opponent = Paper; TargetOutcome = Won } -> Scissors
            | { Opponent = Scissors; TargetOutcome = Lost } -> Paper
            | { Opponent = Scissors; TargetOutcome = Draw } -> Scissors
            | { Opponent = Scissors; TargetOutcome = Won } -> Rock
        buildResult roundStrategy meShape
        

    let rec game (strategy : Part2.RoundStrategy list) : RoundResult list = 
        match strategy with 
        | [] -> []
        | roundStrategy::rest -> 
            (playRound roundStrategy)::(game rest)

In [51]:
let part2SampleResults = Part2.game Part2.sampleStrategy
part2SampleResults

index,Opponent,Me,Outcome
0,Rock,Rock,Draw
1,Paper,Rock,Lost
2,Scissors,Rock,Won


In [52]:
calculateScore part2SampleResults

12

For the actual input:

In [53]:
Part2.actualStrategy
|> Part2.game 
|> calculateScore

15457